#### Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import json

from statsmodels.tsa.stattools import adfuller
from matplotlib import pyplot as plt

from preprocessing.ImputeMean import ImputeMean
from preprocessing.TrainTestSplit import TrainTestSplit
from preprocessing.ZeroSales import ZeroSales
from preprocessing.DataAggregator import DataAggregator
from preprocessing.FeatureEngineering import Lag, Log

from model.Train import SRX, RFR
from model.Optimize import SarimaxHyperopt, RFR_Optuna
from model.Evaluate import Evaluate

from joblib import Parallel, delayed

# Read data
calendar_df = pd.read_csv('E:/Documents/TanXor/Dataset/calendar.csv')
sales_df = pd.read_csv('E:/Documents/TanXor/Dataset/sales_train_validation.csv')
validation_df = pd.read_csv('E:/Documents/TanXor/Dataset/sales_train_evaluation.csv')

#### Data Transformation

In [ ]:
# Initializing Parameters
date = calendar_df['date'].iloc[:1941]
col1, col2 = 'store_id', 'dept_id'

data = DataAggregator(validation_df)

# Takes col1 and col2 and aggregates them into a new column
data.aggregate(col1, col2)

# Drops passed columns
data.drop(col1)

# Groups by the new column
data.group_by()

# Transforms the dataframe using '.T' function
data.transform()

# Sets the index to the date column
data = data.set_datetime_index(date)

# Returns a dataframe with the number of zero sales for each store and department
zero_neg = ZeroSales(data).zero_sales()

# Replace zero sales with the mean of sales of that respective store and department
ImputeMean(data, 0).imputer()

#Split data into evaluation and validation sets
eval_data, val_data = data.iloc[:1913, :], data.iloc[1913:, :]

# Splits the data into train and test sets
train_data, test_data = TrainTestSplit(eval_data, 0.2).data_split()

# Sets the frequency of the data to daily
eval_data.index.freq = val_data.index.freq = train_data.index.freq = test_data.index.freq = 'd'

In [ ]:
### For different datasets with same features ###
'''
df_instances = []
datasets = [sales_df, validation_df]
dates = [calendar_df['date'].iloc[:1913], calendar_df['date'].iloc[:1941]]

for i in range(2):
    df = DataAggregator(datasets[i])
    df.aggregate(col1, col2)
    df.group_by()
    df.transform()
    df.set_datetime_index(dates[i])
    ImputeMean(df.data, 0).imputer()
    df_instances.append(df)

sales, valid_df = df_instances[0].data, df_instances[1].data
val_df = valid_df.iloc[1913:, :]
'''

In [ ]:
def save_to_json(filename, data, mode='w'):
    with open(f"./best_params/{filename}.json", mode) as f:
        json.dump(data, f, indent=4)

def read_from_json(filename,mode="r"):
    with open(f'./best_params/{filename}.json', mode) as f:
        # Load the JSON data into a Python dictionary
        params_data = json.load(f)

        return params_data

In [ ]:
def hyperparameter_tuning(train_data, test_data, col, evals = 3):
    params_dict = [{}, {}]

    srx_hyperopt = SarimaxHyperopt(train_data[col], test_data[col])

    rfr_optuna = RFR_Optuna(train_data[[col]], 7 , 0.2)

    models = [srx_hyperopt, rfr_optuna]

    for i, model in enumerate(models):

        model.hyperparameter_tune(evals)

        params_dict[i][col] = model.best_params

    return params_dict

In [ ]:
output = [{}, {}]

for i, col in enumerate(train_data.columns[:2]):
    output[i] = hyperparameter_tuning(train_data, test_data, col, 2)

In [ ]:
output = Parallel(n_jobs=-1)(
delayed(hyperparameter_tuning)(train_data, test_data, col, 2)
for col in train_data.columns[:20])

In [ ]:
srx_best_ = output[0][0]
rfr_best_ = output[0][1]

for i in range(1, len(output)):
    srx_best_.update(output[i][0])
    rfr_best_.update(output[i][1])

params_path = ['sarimax_best_params', 'rfr_best_params']
params_data = [srx_best_, rfr_best_]

for i, path in enumerate(params_path):
    save_to_json(path, params_data[i], 'w')

In [ ]:
srx_params = {}
rfr_params = {}

params = [srx_params, rfr_params]
paths = ['sarimax_best_params', 'rfr_best_params']
model_names = ['SARIMAX', 'Random_Forest']

for i, path in enumerate(paths):
    params[i] =  read_from_json(path, mode="r")

In [ ]:
def model_training(eval_data, val_data, data, col, params, model_names, total_forecast):

    total_forecast[f'Original_{col}'] = val_data[col]

    srx_model = SRX(eval_data[col], val_data[[col]])

    rfr_model = RFR(data[[col]])

    rfr_model.data_preprocess(7)

    rfr_model.train_test_split(len(val_data)/len(data))

    models = [srx_model, rfr_model]

    for i, model in enumerate(models):

        model.fit(params[i][col])

        forecast = model.predict()

        total_forecast[f'{model_names[i]}_{col}'] = forecast
    
    return total_forecast

In [ ]:
forecast = pd.DataFrame()

for col in train_data.columns[:2]:
    forecast = model_training(eval_data, val_data, data, col, params, model_names, forecast)

In [ ]:
forecast = pd.DataFrame()

forecast = Parallel(n_jobs=-1)(
delayed(model_training)(eval_data, val_data, data, col, params, model_names, forecast)
for col in train_data.columns[:2])

forecast = pd.concat(forecast, axis=1)

In [ ]:
sarimax_forecast = forecast[[col for col in forecast.columns if "SARIMAX" in col]].copy()
randomforest_forecast = forecast[[col for col in forecast.columns if "Random" in col]].copy()

sarimax_forecast['Total'] = sarimax_forecast.sum(axis=1)
randomforest_forecast['Total'] = randomforest_forecast.sum(axis=1)

forecast.to_csv("E:/Documents/TanXor/Model/forecasted_data/total_forecast.csv", header=True)
sarimax_forecast.to_csv("E:/Documents/TanXor/Model/forecasted_data/sarimax_forecast.csv", header=True)
randomforest_forecast.to_csv("E:/Documents/TanXor/Model/forecasted_data/randomforest_forecast.csv", header=True)

In [ ]:
sarimax_forecast = pd.read_csv('E:/Documents/TanXor/Dataset/forecasted_data/sarimax_forecast.csv')
randomforest_forecast = pd.read_csv('E:/Documents/TanXor/Dataset/forecasted_data/randomforest_forecast.csv')

model_eval = Evaluate(val_data.iloc[:,:2].sum(axis = 1))

predictions = [sarimax_forecast['Total'], randomforest_forecast['Total']]

for pred in predictions:
    print(model_eval.mape(pred))

In [ ]:
srx_best_ = {}
rfr_best_ = {}

best_params_data = [srx_best_, rfr_best_]

params_path = ['sarimax_best_params', 'rfr_best_params']

for col in train_data.columns[:2]:

    srx_hyperopt = SarimaxHyperopt(train_data[col], test_data[col])

    rfr_optuna = RFR_Optuna(train_data[[col]], 7 , 0.2)

    models = [srx_hyperopt, rfr_optuna]

    for i, model in enumerate(models):

        model.hyperparameter_tune(2)

        best_params_data[i][col] = model.best_params

for i, path in enumerate(params_path):
    save_to_json(path, best_params_data[i], 'w')

In [ ]:
'''
import dask
from dask import delayed

## iterate over values and calculate the sum
for i, col in enumerate(train_data.columns[:2]):
    result = delayed(hyperparameter_tuning)(train_data, test_data, col, 2)
    out = result.compute()
'''

### Single Model Algorithm

In [ ]:
best_params_data = {}

for i in train_data.columns[:2]:

    hp_model = SarimaxHyperopt(train_data[i], test_data[i])

    hp_model.hyperparameter_tune(num_evals=3)

    best = hp_model.best_params

    input_data = {i: best}

    best_params_data.update(input_data)

save_to_json('sarimax_best_params', best_params_data, 'w')

In [ ]:
best_params = read_from_json('sarimax_best_params', mode="r")
total_forecast = pd.DataFrame()

for i in train_data.columns[:2]:
    model = SRX(train_data[i])

    model.train(best_params[i])

    forecast = model.predict(val_df[[i]])

    total_forecast[f'Original_{i}'] = val_df[i]

    total_forecast[f'Forecast_{i}'] = forecast.values

total_forecast.head()

### Feature Engineering

In [ ]:
# Creates a new Dataframe with the following columns:
# 1. Date, 2. Actual, 3. Lagged
seasonal_lag = Lag(train_data[[train_data.columns[2]]]).lag_transform(7, train_data.columns[2])

# Creates a new Dataframe with the following columns:
# 1. Date, 2. Actual, 3. LogTransformed
log_transform = Log(train_data).log_transform(train_data.columns[0])

In [ ]:
# Differencing
# Adds a column to the dataframe that is the difference between the actual and lagged data
seasonal_lag['Seasonal_Diff'] = seasonal_lag.iloc[:,0] - seasonal_lag.iloc[:,1]
seasonal_lag = seasonal_lag.dropna()
seasonal_lag

### Stationarity Test

In [ ]:
# Augmented Dickey-Fuller Test (Stationarity Test)
def adfuller_test(data):
    result=adfuller(data)
    return result[1] # Return p-value

# Testing for stationarity on all columns
for i in train_data.columns[:70]:
    if (adfuller_test(train_data[i]) > 0.05):
        print(i, "Fail")

        seasonal_lag = Lag(train_data[[i]]).lag_transform(7, i)
        seasonal_lag['Seasonal_Diff'] = seasonal_lag.iloc[:,0] - seasonal_lag.iloc[:,1]
        seasonal_lag = seasonal_lag.dropna()
        
        if (adfuller_test(seasonal_lag.iloc[:,-1]) > 0.05):
            print(i, "Failed Again")
        else:
            print(i, "Passed")       
        

## Model Training

### SARIMAX

In [ ]:
# Set model Parameters
params = {
    'p': 1, 'd': 1, 'q': 4,
    'P': 2, 'D': 1, 'Q': 3, 'm': 7
}

col = test_data.columns[0]

In [ ]:
# Initialize the model with the training data
model = SRX(train_data[col])

# Train the model on the given parameters
model.train(params)

# Predict the values using the model
test_data['forecast']=model.predict(test_data[col])

# Evaluate the model on the above predictions
print(model.evaluate())

# Plot the forecast against the actuals
test_data[[col, 'forecast']].plot()

### Random Forest Regressor

In [ ]:
rfr_params = {
    'n_estimators': 100,
    'max_depth': 25,
    'max_features':'log2',
    'min_samples_leaf': 10,
    'min_samples_split': 12,
    'bootstrap': False,
}

col = test_data.columns[0]

In [ ]:
model = RFR(train_data[[col]])

model.data_preprocess(7)

model.train_test_split(0.2)

model.fit(rfr_params)

model.predict()

rfr_mape = model.evaluate()

print(f'MAPE: {rfr_mape:.2f}%')

## Hyperparameter Tuning

### SARIMAX_HYPEROPT

In [ ]:
# Initalize the model with the test and train data:
hp_model = SarimaxHyperopt(train_data[train_data.columns[0]], test_data[test_data.columns[0]])

# Tune the hyperparameters:
hp_model.hyperparameter_tune(num_evals=3)

# Returns the best parameters for the model
best = hp_model.best_params
print('Best parameters for the model: ', best)

### Random Forest Regressor Optuna

In [ ]:
# Initalize the model with the unprocessed raw data:
model = RFR_Optuna(train_data[[col]])

# Add new lagged columns to the data:
model.data_preprocess(7)

# Create the features and target:
model.train_test_split(0.2)

# Hyperparameter tuning:
model.hyperparameter_tune()

# Train the model:
model.fit()

# Make predictions:
model.predict()

# Evaluate the model:
rfr_mape = model.evaluate()

print(f'MAPE: {rfr_mape:.2f}%')

In [ ]:
pred = mod.predict(X_test)
plt.rcParams["figure.figsize"] = (12,8)
plt.plot(pred,label='Random_Forest_Predictions')
plt.plot(y_test,label='Actual Sales')
plt.legend(loc="upper left")
plt.show()

### Custom Methods

In [ ]:
### Custom Hyperparameter Tuning ###
'''
import itertools

### Define Parameter Ranges to Test ###

# Note: higher numbers will result in code taking much longer to run
# Here we have it set to test p,d,q each = 0, 1 & 2

# Define the p, d and q parameters to take any value between 0 and 3 (exclusive)
p = range(1, 6)
q = range(0, 6)
d = range(1, 2)
P = range(0, 4)
Q = range(0, 4)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
# Note: here we have 12 in the 's' position as we have monthly data
# You'll want to change this according to your time series' frequency
pdqs = [(x[0], x[1], x[2], 7) for x in list(itertools.product(P, d, Q))]

### Run Grid Search ###

# Note: this code will take a while to run

# Define function
def sarimax_gridsearch(ts, pdq, pdqs, freq='D'):
    
    Input: 
        ts : your time series data
        pdq : ARIMA combinations from above
        pdqs : seasonal ARIMA combinations from above
        maxiter : number of iterations, increase if your model isn't converging
        frequency : default='M' for month. Change to suit your time series frequency
            e.g. 'D' for day, 'H' for hour, 'Y' for year. 
        
    Return:
        Prints out top 5 parameter combinations
        Returns dataframe of parameter combinations ranked by BIC
    

    # Run a grid search with pdq and seasonal pdq parameters and get the best BIC value
    ans = []
    for comb in pdq:
        for combs in pdqs:
            try:
                mod = sm.tsa.statespace.SARIMAX(ts, # this is your time series you will input
                                                order=comb,
                                                seasonal_order=combs,
                                                enforce_stationarity=False, 
                                                enforce_invertibility=False,
                                                freq=freq)

                output = mod.fit(maxiter=1000)
                predictions = output.predict(start=1800,end=1913,dynamic=True)

                test_data = ts.iloc[1800:1913]
                mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

                ans.append([comb, combs, output.bic, mape])
                print('SARIMAX {} x {}12 : MAPE Calculated ={}'.format(comb, combs, mape))
            except:
                continue
            
    # Find the parameters with minimal BIC value

    # Convert into dataframe
    ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'bic', 'mape'])

    # Sort and return top 5 combinations
    ans_df = ans_df.sort_values(by=['mape'],ascending=True)
    
    return ans_df
    

    ### Apply function to your time series data ###
#Remember to change frequency to match your time series data
best_params = sarimax_gridsearch(df_1['Sales'], pdq, pdqs, freq='D')
best_params.head(20)
'''

In [ ]:
### Optuna Hyperparameter Optimization ###
'''
def objective(trial):
    p = trial.suggest_int('p', 0, 6)
    # d = trial.suggest_int('d', 1, 3)
    q = trial.suggest_int('q', 0, 6)
    P = trial.suggest_int('P', 0, 6)
    Q = trial.suggest_int('Q', 0, 6)
    # m = trial.suggest_int('m', 3, 8)
    srx = sm.tsa.statespace.SARIMAX(df_1['Sales'], 
                                    order=(p,1,q), 
                                    seasonal_order=(P,1,Q,7), 
                                    enforce_stationarity=False, 
                                    enforce_invertibility=False,
                                    freq='D')
    
    output = srx.fit(maxiter=1000)
    predictions = output.predict(start=1800,end=1913,dynamic=True)

    test_data = df_1['Sales'].iloc[1800:1913]
    mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

    return mape

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
'''